# Chapter 10

In [1]:
import pandas as pd
import numpy as np

## 10.1   GroupBy Mechanics

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 
                   'key2' : ['one', 'two', 'one', 'two', 'one'], 
                   'data1' : np.random.randn(5), 
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.775270,0.343707
1,a,two,1.890696,-0.781609
2,b,one,0.347403,0.253704
3,b,two,0.897549,-1.255243
4,a,one,-0.231099,-1.624076


Suppose you wanted to compute the mean of the `data1` column using the labels from `key1`. There are a number of ways to do this. One is to access `data1` and call `groupby` with the column (a Series) at `key1`:

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

This `grouped` variable is now a *GroupBy* object. It has not actually computed anything yet except for some intermediate data about the group key `df['key1']`.

In [4]:
grouped.mean()

key1
a    0.294776
b    0.622476
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.503185
      two     1.890696
b     one     0.347403
      two     0.897549
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.503185,1.890696
b,0.347403,0.897549


In this example, the group keys are all Series, though they could be any arrays of the right length:

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    1.890696
            2006    0.347403
Ohio        2005    0.061139
            2006   -0.231099
Name: data1, dtype: float64

Frequently the grouping information is found in the same DataFrame as the data you want to work on. In that case, you can pass column names (whether those are strings, numbers, or other Python objects) as the group keys:

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.294776,-0.687326
b,0.622476,-0.500770


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.503185 -0.640184
     two   1.890696 -0.781609
b    one   0.347403  0.253704
     two   0.897549 -1.255243

You may have noticed in the first case `df.groupby('key1').mean()` that there is no `key2` column in the result. Because `df['key2']` is not numeric data, it is said to be a *nuisance column*, which is therefore excluded from the result. By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset.

Regardless of the objective in using `groupby`, a generally useful GroupBy method is `size`, which returns a Series containing group sizes:

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

Take note that any missing values in a group key will be *excluded* from the result.

### Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.

In [15]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.775270  0.343707
1    a  two  1.890696 -0.781609
4    a  one -0.231099 -1.624076
b
  key1 key2     data1     data2
2    b  one  0.347403  0.253704
3    b  two  0.897549 -1.255243


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.775270  0.343707
4    a  one -0.231099 -1.624076
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.890696 -0.781609
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.347403  0.253704
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.897549 -1.255243


In [18]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.347403,0.253704
3,b,two,0.897549,-1.255243


By default `groupby` groups on `axis=0`, but you can group on any of the other axes.

In [19]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.775270  0.343707
1  1.890696 -0.781609
2  0.347403  0.253704
3  0.897549 -1.255243
4 -0.231099 -1.624076
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [22]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.503185 -0.640184
     two   1.890696 -0.781609
b    one   0.347403  0.253704
     two   0.897549 -1.255243

In [23]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.640184
     two  -0.781609
b    one   0.253704
     two  -1.255243

The object returned by this indexing operation is a grouped DataFrame if a list or array is passed or a grouped Series if only a single column name is passed as a scalar:

In [24]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [25]:
s_grouped.mean()

key1  key2
a     one    -0.640184
      two    -0.781609
b     one     0.253704
      two    -1.255243
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [26]:
people = pd.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.591521,1.075474,1.177439,-1.177001,-1.835243
Steve,1.475318,1.374961,-0.950910,-1.605346,-1.297425
Wes,-2.321072,NaN,NaN,1.349623,1.980524
Jim,0.398817,-1.315653,1.006149,-0.373739,-0.326115
Travis,-1.065284,0.850750,-0.626660,-1.667156,-0.181501


In [27]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.000438,-0.168248
Steve,-2.556256,1.552854
Wes,1.349623,-0.340548
Jim,0.632410,-1.242951
Travis,-2.293816,-0.396035


The same functionality holds for Series, which can be viewed as a fixed-size mapping:

In [28]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [29]:
people.groupby(map_series, axis=1).sum()

,blue,red
Joe,0.000438,-0.168248
Steve,-2.556256,1.552854
Wes,1.349623,-0.340548
Jim,0.632410,-1.242951
Travis,-2.293816,-0.396035


### Grouping with Functions

In [30]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.330733,-0.240179,2.183587,-0.201116,-0.180834
5,1.475318,1.374961,-0.950910,-1.605346,-1.297425
6,-1.065284,0.850750,-0.626660,-1.667156,-0.181501


Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:

In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -2.321072  1.075474  1.177439 -1.177001 -1.835243
  two  0.398817 -1.315653  1.006149 -0.373739 -0.326115
5 one  1.475318  1.374961 -0.950910 -1.605346 -1.297425
6 two -1.065284  0.850750 -0.626660 -1.667156 -0.181501

### Grouping by Index Levels

In [32]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.934181 -1.100239  0.852168  1.416545  0.850797
1      0.040116  0.559248 -1.171504  1.941032  0.305427
2      0.682838  1.678858 -1.223492 -0.269841 -1.304505
3      0.301744  1.544566 -0.755317 -0.470639  1.531633

To group by level, pass the level number or name using the `level` keyword:

In [33]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2   Data Aggregation

Aggregations refer to any data transformation that produces scalar values from arrays.

In [34]:
df

,key1,key2,data1,data2
0,a,one,-0.775270,0.343707
1,a,two,1.890696,-0.781609
2,b,one,0.347403,0.253704
3,b,two,0.897549,-1.255243
4,a,one,-0.231099,-1.624076


In [35]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.466337
b    0.842534
Name: data1, dtype: float64

To use your own aggregation functions, pass any function that aggregates an array to the `aggregate` or `agg` method:

In [36]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [40]:
# grouped.agg(peak_to_peak)   # deprecated -- select only columns which should be valid for the aggregating function
# grouped.aggregate(peak_to_peak)   # deprecated -- select only columns which should be valid for the aggregating function
grouped[['data1', 'data2']].agg(peak_to_peak)

,data1,data2
key1,,
a,2.665966,1.967782
b,0.550146,1.508947


You may notice that some methods like `describe` also work, even though they are not aggregations, strictly speaking:

In [42]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.294776  1.408635 -0.775270 -0.503185 -0.231099  0.829798   
b      2.0  0.622476  0.389012  0.347403  0.484939  0.622476  0.760012   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.890696   3.0 -0.687326  0.987273 -1.624076 -1.202842 -0.781609   
b     0.897549   2.0 -0.500770  1.066986 -1.255243 -0.878006 -0.500770   

                          
           75%       max  
key1                      
a    -0.218951  0.343707  
b    -0.123533  0.253704

### Column-Wise and Multiple Function Application

In [43]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [46]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']

In [47]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [48]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

You don’t need to accept the names that GroupBy gives to the columns; notably, `lambda` functions have the name `'<lambda>'`, which makes them hard to identify (you can see for yourself by looking at a function’s `__name__` attribute). Thus, if you pass a list of `(name, function)` tuples, the first element of each tuple will be used as the DataFrame column names (you can think of a list of 2-tuples as an ordered mapping):

In [50]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame you have more options, as you can specify a list of functions to apply to all of the columns or different functions per column.

In [52]:
functions = ['count', 'mean', 'max']
# result = grouped['tip_pct', 'total_bill'].agg(functions)   # deprecated -- indexing with multiple keys will be deprecated
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [53]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [54]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Suppose you want to apply potentially different functions to one or more of the columns. To do this, pass a dict to `agg` that contains a mapping of column names to any of the function specifications listed so far:

In [55]:
grouped.agg({'tip': np.max, 'size': 'mean'})

tip      size
day  smoker                 
Fri  No       3.50  2.250000
     Yes      4.73  2.066667
Sat  No       9.00  2.555556
     Yes     10.00  2.476190
Sun  No       6.00  2.929825
     Yes      6.50  2.578947
Thur No       6.70  2.488889
     Yes      5.00  2.352941

In [56]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size': 'mean'})

tip_pct                                    size
                  min       max      mean       std      mean
day  smoker                                                  
Fri  No      0.120385  0.187735  0.151650  0.028123  2.250000
     Yes     0.103555  0.263480  0.174783  0.051293  2.066667
Sat  No      0.056797  0.291990  0.158048  0.039767  2.555556
     Yes     0.035638  0.325733  0.147906  0.061375  2.476190
Sun  No      0.059447  0.252672  0.160113  0.042347  2.929825
     Yes     0.065660  0.710345  0.187250  0.154134  2.578947
Thur No      0.072961  0.266312  0.160298  0.038774  2.488889
     Yes     0.090014  0.241255  0.163863  0.039389  2.352941

### Returning Aggregated Data Without Row Indexes

In [57]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


Of course, it’s always possible to obtain the result in this format by calling `reset_index` on the result. Using the `as_index=False` method avoids some unnecessary computations.

## 10.3   Apply: General split-apply-combine

In [58]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [59]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

The `top` function is called on each row group from the DataFrame, and then the results are glued together using `pandas.concat`, labeling the pieces with the group names. The result therefore has a hierarchical index whose inner level contains index values from the original DataFrame.

If you pass a function to `apply` that takes other arguments or keywords, you can pass these after the function:

In [60]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

Call `describe` on a GroupBy object:

In [62]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [63]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside GroupBy, when you invoke a method like `describe`, it is actually just a short‐cut for:

In [64]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Suppressing the Group Keys

In [65]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [66]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

In [67]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.788, -0.0731]
1    (-1.788, -0.0731]
2      (-3.51, -1.788]
3      (-3.51, -1.788]
4    (-1.788, -0.0731]
5     (-0.0731, 1.642]
6    (-1.788, -0.0731]
7     (-0.0731, 1.642]
8     (-0.0731, 1.642]
9     (-0.0731, 1.642]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.51, -1.788] < (-1.788, -0.0731] < (-0.0731, 1.642] < (1.642, 3.357]]

The `Categorical` object returned by `cut` can be passed directly to `groupby`. So we could compute a set of statistics for the `data2` column like so:

In [69]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.51, -1.788]",-1.934736,1.996492,38.0,-0.265682
"(-1.788, -0.0731]",-2.960244,2.562427,445.0,0.004416
"(-0.0731, 1.642]",-2.999267,3.757822,467.0,-0.033066
"(1.642, 3.357]",-1.945785,2.191243,50.0,0.047334


These were equal-length buckets; to compute equal-size buckets based on sample quantiles, use `qcut`. I’ll pass `labels=False` to just get quantile numbers:

In [71]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-1.934736,2.110678,100.0,-0.057442
1,-2.960244,2.562427,100.0,0.076309
2,-1.771463,2.047201,100.0,0.102887
3,-2.263626,1.655096,100.0,-0.194772
4,-2.580559,1.846043,100.0,0.032906
5,-1.913032,2.542809,100.0,-0.019995
6,-2.999267,1.930980,100.0,-0.173016
7,-1.860146,3.757822,100.0,0.059911
8,-2.574373,3.262079,100.0,-0.099815


### Example: Filling Missing Values with Group-Speciic Values

In [72]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    1.873749
2         NaN
3    0.637314
4         NaN
5   -0.562593
dtype: float64

In [73]:
s.fillna(s.mean())

0    0.649490
1    1.873749
2    0.649490
3    0.637314
4    0.649490
5   -0.562593
dtype: float64

Suppose you need the fill value to vary by group. One way to do this is to group the data and use `apply` with a function that calls `fillna` on each data chunk.

In [76]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.495806
New York     -0.276160
Vermont            NaN
Florida      -0.756624
Oregon        1.850584
Nevada             NaN
California    0.792399
Idaho              NaN
dtype: float64

In [77]:
data.groupby(group_key).mean()

East   -0.509530
West    1.321491
dtype: float64

In [78]:
fill_mean = lambda g: g.fillna(g.mean())

In [79]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.495806
New York     -0.276160
Vermont      -0.509530
Florida      -0.756624
Oregon        1.850584
Nevada        1.321491
California    0.792399
Idaho         1.321491
dtype: float64

In another case, you might have predefined fill values in your code that vary by group. Since the groups have a `name` attribute set internally, we can use that:

In [81]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.495806
New York     -0.276160
Vermont       0.500000
Florida      -0.756624
Oregon        1.850584
Nevada       -1.000000
California    0.792399
Idaho        -1.000000
dtype: float64

### Example: Group Weighted Average and Correlation

In [82]:
close_px = pd.read_csv('stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [83]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


One task of interest might be to compute a DataFrame consisting of the yearly correlations of daily returns (computed from percent changes) with `SPX`.

In [84]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()

In [85]:
get_year = lambda x: x.year
by_year = close_px.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.882692,0.675932,0.860104,1.0
2004,0.638080,0.593300,0.432642,1.0
2005,0.743263,0.746188,0.404042,1.0
2006,0.838252,0.824727,0.849528,1.0
2007,0.483093,0.292706,0.729426,1.0
2008,0.829417,0.919476,0.774885,1.0
2009,0.952895,0.950194,0.278447,1.0
2010,0.617886,0.469280,0.894656,1.0
2011,-0.552395,0.111688,0.912603,1.0


In [87]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.666938
2004    0.784780
2005    0.570353
2006    0.815130
2007    0.761417
2008    0.729151
2009    0.951722
2010   -0.342094
2011    0.307734
dtype: float64

### Example: Group-Wise Linear Regression

In [88]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [89]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,0.018506,-8.589850
2004,0.132565,-132.121163
2005,0.286831,-299.595103
2006,0.155668,-133.186913
2007,0.379906,-432.917550
2008,0.146157,-36.338143
2009,0.328253,-164.384872
2010,0.412900,-210.849814
2011,-0.193834,603.876911


## 10.4   Pivot Tables and Cross-Tabulation

In [90]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

Suppose we want to aggregate only `tip_pct` and `size`, and additionally group by `time`. I’ll put `smoker` in the table columns and `day` in the rows:

In [91]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

We could augment this table to include partial totals by passing `margins=True`. This has the effect of adding `All` row and column labels, with corresponding values being the group statistics for all the data within a single tier:

In [94]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

Here, the `All` values are means without taking into account smoker versus non-smoker (the `All` columns) or any of the two levels of grouping on the rows (the `All` row).

To use a different aggregation function, pass it to `aggfunc`. For example, `'count'` or `len` will give you a cross-tabulation (count or frequency) of group sizes:

In [97]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

If some combinations are empty (or otherwise NA), you may wish to pass a `fill_value`:

In [98]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True, fill_value=0)

day            Fri  Sat  Sun  Thur  All
time   smoker                          
Dinner No        3   45   57     1  106
       Yes       9   42   19     0   70
Lunch  No        1    0    0    44   45
       Yes       6    0    0    17   23
All             19   87   76    62  244

### Cross-Tabulations: Crosstab

A cross-tabulation (or *crosstab* for short) is a special case of a pivot table that computes group frequencies.

In [99]:
data = pd.DataFrame({'Sample': range(1, 11), 
                     'Nationality': ['USA', 'Japan', 'USA', 'Japan', 'Japan', 'Japan', 'USA', 'USA', 'Japan', 'USA'], 
                     'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 
                                    'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed']})
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [100]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


The first two arguments to `crosstab` can each either be an array or Series or a list of arrays. As in the tips data:

In [101]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244